In [1]:
import tensorflow as tf
import numpy as np
import nibabel as nib
import glob
import matplotlib.pyplot as plt
import nibabel as nib

In [2]:
from BSUnetModified import *

Using TensorFlow backend.


In [3]:
num_of_epochs = 1
global_best_metric = 0
def read_ct(path):
    img = nib.load(path)
    img = img.get_data()
    return img
def loadCT(path):
    images = glob.glob(path+"/volume*")
    segmentations = glob.glob(path+"/segmentation*", )
    images = sorted(images)
    segmentations = sorted(segmentations)
    return images , segmentations
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [8]:
def trainUnet(model,model_checkpoint,num_channels=2,num_ct=1,folders=2,batch_size=8):
    """
        Training by taking ct scans of only num_ct files and each data point of shape
        (512,512,num_channels)
    """
    path = "../data/batch"
    images ,segmentations = loadCT(path)
    for i in range(0,len(images),num_ct):
        print("image " + str(i)+" out of "+str(len(images)))
        X_train = []
        y_train = []
        img = read_ct(images[i])
        seg = read_ct(segmentations[i])
        print("Shape of img : ", img.shape)
        ##img shape: (512,512,X) X is the sum of all slices of num_ct files
        for j in range(0,img.shape[2]):
            ## simg shape (512,512)
#             simg = img[:,:,j].astype(float)
            sseg = seg[:,:,j]
            ##HU clipping
#             simg[simg >250 ] = 250
#             simg[simg < -200] = -200
#             ## Normalization
#             simg -= -200
#             simg /= 450
            ## treating tumor as part of liver
            sseg[sseg > 0] = 1
            if np.sum(sseg == 1)>0 :
                X_train.append(sseg)
                y_train.append(sseg)
        print("Len of X_train ",len(X_train))
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_train = X_train[...,np.newaxis]
        y_train = y_train[...,np.newaxis]
#         mean = np.mean(X_train)  # mean for data centering
#         std = np.std(X_train)  # std for data normalization

#         X_train -= mean
#         X_train /= std
        print("shape of X_train ",X_train.shape)
        print("Shape of y_train ",y_train.shape)
        model.fit(X_train,[y_train,np.zeros((y_train.shape[0],16,16,128), dtype=int)],callbacks=[model_checkpoint],batch_size=batch_size) ## set epoch to 1
    return model

In [9]:
def evaluate(model,fromIndex,batch_size=8):
    path = "../data/Test"
    images ,segmentations = loadCT(path)
    histot = []
    for i in range(fromIndex,len(images)):
        print("image " + str(i))
        X_test = []
        y_test = []
        img = read_ct(images[i])
        seg = read_ct(segmentations[i])
        print("Shape of img : ", img.shape)
        ##img shape: (512,512,X) X is the sum of all slices of num_ct files
        for j in range(0,img.shape[2]):
#             simg = img[:,:,j].astype(float)
            sseg = seg[:,:,j]
            ## simg shape (512,512)
            ##HU clipping
#             simg[simg >250 ] = 250
#             simg[simg < -200] = -200
#             ## Normalization
#             simg -= -200
#             simg /= 450
            ## treating tumor as part of liver
            sseg[sseg > 0] = 1
            if np.sum(sseg == 1)>0 :
                X_test.append(sseg)
                y_test.append(sseg)
        print("Len of X_test ",len(X_test))
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        X_test = X_test[...,np.newaxis]
        y_test = y_test[...,np.newaxis]
#         mean = np.mean(X_test)  # mean for data centering
#         std = np.std(X_test)  # std for data normalization

#         X_test -= mean
#         X_test /= std
        print("shape of X_train ",X_test.shape)
        print("Shape of y_train ",y_test.shape)
        history = model.evaluate(X_test,y_test,batch_size=batch_size)
        print(history)
        histot.append(history)
    return histot

In [10]:
num_channels = 1
num_ct = 1
# model = liverUnet(input_size=(512,512,num_channels))
# model = get_unet_sorr(input_size=(512,512,num_channels))
model = baseUNet(input_size=(512,512,num_channels),output_ch=(512,512,num_channels))
model_checkpoint = ModelCheckpoint('./weights/AutoencoderBSUnet/best_weights.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 512, 512, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 512, 512, 8)  16          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 512, 512, 8)  16          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_52 

In [11]:
num_epochs = 2
for e in range(num_epochs):
    print("*"*50)
    print("** epoch ",e)
    model = trainUnet(model,model_checkpoint,num_channels=num_channels,num_ct=num_ct,folders=1,batch_size=10)
    model.save_weights('./weights/AutoencoderBSUnet/after_epoch{}.hdf5'.format(e))
    model.save_weights('./weights/AutoencoderBSUnet/final_weights.hdf5')

**************************************************
** epoch  0
image 0 out of 111
Shape of img :  (512, 512, 685)
Len of X_train  276
shape of X_train  (276, 512, 512, 1)
Shape of y_train  (276, 512, 512, 1)
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor of shape [3,3,1024,256] and type float
	 [[Node: training_1/Adam/zeros_232 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,3,1024,256] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training_1/Adam/zeros_232', defined at:
  File "/home/arshdeep/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/arshdeep/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/arshdeep/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/arshdeep/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/arshdeep/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/arshdeep/anaconda3

# Validation

In [ ]:
# model.load_weights('unet_liver_preprocess.hdf5')
# model.load_weights('unet_liver_after_epoch.hdf5')
h = evaluate(model,0,batch_size=2)

In [ ]:
alp = np.array(h)
y = np.mean(alp,axis=0)
print(y)
print(global_best_metric)
# if y[1]>global_best_metric[1]:
#     global_best_metric = y

In [ ]:
# model.load_weights('unet_liver_after_epoch.hdf5')

# Testing

In [ ]:
images ,segmentations = loadCT("../data/Test")
img = read_ct(images[11])
seg = read_ct(segmentations[11])
X_test = []
y_test = []
for j in range(0,img.shape[2]):
    simg = img[:,:,j].astype(float)
    sseg = seg[:,:,j]
    ## simg shape (512,512)
    ##HU clipping
    simg[simg >250 ] = 250
    simg[simg < -200] = -200
    ## Normalization
    simg -= -200
    simg /= 450
    ## treating tumor as part of liver
    sseg[sseg > 0] = 1
    if np.sum(sseg == 1)>0 :
        X_test.append(simg)
        y_test.append(sseg)
print("Len of X_test ",len(X_test))
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = X_test[...,np.newaxis]
y_test = y_test[...,np.newaxis]
print("shape of X_train ",X_test.shape)
print("Shape of y_train ",y_test.shape)
# history = model.evaluate(X_test,y_test,batch_size=batch_size)


In [ ]:
idx = 100

In [ ]:
plt.imshow(X_test[idx][:,:,0].reshape(512,512), cmap="gray")
plt.imshow(y_test[idx][:,:,0].reshape(512,512), cmap='jet', alpha=0.5)

In [ ]:
result = model.predict(X_test[idx:idx+1])

In [ ]:
result.shape

In [ ]:
plt.imshow(X_test[idx][:,:,0].reshape(512,512), cmap="gray")
plt.imshow(result[0][:,:,0].reshape(512,512), cmap='jet', alpha=0.5)

In [ ]:
np.sum(result>0)